# Qdrant 101

![qdrant](https://qdrant.tech/images/logo_with_text.png)

Vector databases are a relatively new way for interacting with abstract data representations derived from opaque machine learning models such as deep learning architectures. These representations are often called vectors or embeddings and they are a compressed version of the data used to train a machine learning model to accomplish a task like sentiment analysis, speech recognition, object detection, and many others.

These new databases shine in many applications like [semantic search](https://en.wikipedia.org/wiki/Semantic_search) and [recommendation systems](https://en.wikipedia.org/wiki/Recommender_system), and in this tutorial, we'll learn about how to get started with one of the most popular and fastest growing vector databases in the market, [Qdrant](qdrant.tech).

## Table of Contents

1. Learning Outcomes
2. What is Qdrant?
    - What are Vector Databases?
    - Why do We Need Vector Databases??
    - Overview of Qdrant's Architecture    
    - Installation
3. Getting Started
    - Adding Points
    - Payload
    - Search
4. Recommendations
5. Conclusion
6. Resources

## 1. Learning Outcomes

By the end of this tutorial, you will be able
- Describe what vector databases are and what are they used for.
- Create, update, and query collections of vectors using Qdrant.
- Conduct semantic search based on new data.
- Understand the mechanics the recommendation engine of Qdrant.

## 2. What is Qdrant?

[Qdrant](qdrant.tech) "is a vector similarity search engine that provides a production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload." You can think of the payloads as additional pieces of information that can help you hone in on your search while also returning useful information to your users (we'll talk more about the payload functionality in a bit).

You can get started using Qdrant with the Python `qdrant-client`, by pulling the latest docker image of `qdrant` and connecting to it locally, or by trying out Qdrant's Cloud free tier option until you are ready to make the full switch.

With that out of the way, let's talk about what are vector databases.

### 2.1 What Are Vector Databases?

![dbs](../images/databases.png)

Vector databases are a type of database designed to store and query high-dimensional vectors efficiently. In traditional [OLTP](https://www.ibm.com/topics/oltp) and [OLAP](https://www.ibm.com/topics/olap) databases (as seen in the image above), data is organized in rows and columns, and queries are performed based on the values in those columns. However, in certain applications including image recognition, natural language processing, and recommendation systems, data is often represented as vectors in a high-dimensional space, and these vectors, plus an id and a payload, are the elements we store in a vector database like Qdrant.

A vector in this context is a mathematical representation of an object or data point, where each element of the vector corresponds to a specific feature or attribute of the object. For example, in an image recognition system, a vector could represent an image, with each element of the vector representing a pixel value or a descriptor/characteristic of that pixel.

Vector databases are optimized for **storing** and **querying** these high-dimensional vectors efficiently, often using specialized data structures and indexing techniques such as Hierarchical Navigable Small World (HNSW) -- which is used to implement Approximate Nearest Neighbors -- and Product Quantization, among others. These databases enable fast similarity and semantic search while allowing users to find vectors that are the closest to a given query vector based on some distance metric. The most commonly used distance metrics are Euclidean Distance, Cosine Similarity, and Dot Product.

Now that we know what vector databases are, and how they are structurally different than other databases, let's go over why they are important.

### 2.2 Why do we need Vector Databases?

Vector databases play a crucial role in various applications that require similarity search, such as recommendation systems, content-based image retrieval, and personalized search. By taking advantage of their efficient indexing and searching techniques, vector databases enable faster and more accurate retrieval of similar vectors, which helps advance data analysis and decision-making.

In addition, other benefits of using vector databases include:
1. Efficient storage and indexing of high-dimensional data.
3. Ability to handle large-scale datasets with billions of data points.
4. Support for real-time analytics and queries.
5. Ability to handle vectors derived from complex data types such as images, videos, and natural language text.
6. Improved performance and reduced latency in machine learning and AI applications.
7. Reduced development and deployment time and cost compared to building a custom solution.

Keep in mind that the specific benefits of using a vector database may vary depending on the use case of your organization and the features of the database you ultimately choose.

Let's now evaluate, at a high-level, the way Qdrant is architected.

### 2.3 Overview of Qdrant's Architecture (High-Level)

![qdrant](../images/qdrant_overview_high_level.png)

The diagram above represents a high-level overview of some of the main components of Qdrant. Here are the terminologies you should get familiar with.

- [Collections](https://qdrant.tech/documentation/collections/): A collection is a named set of points (vectors with a payload) among which you can search. Vectors within the same collection can have different dimensionalities and be compared by a single metric.
- [Distance Metrics](https://en.wikipedia.org/wiki/Metric_space): These are used to measure similarities among vectors and they must be selected at the same time you are creating a collection. The choice of metric depends on the way the vectors were obtained and, in particular, on the neural network that will be used to encode new queries.
- [Points](https://qdrant.tech/documentation/points/): The points are the central entity that Qdrant operates with and they consist of a vector and an optional id and payload.
    - id: a unique identifier for your vectors.
    - Vector: a high-dimensional representation of data, for example, an image, a sound, a document, a video, etc.
    - [Payload](https://qdrant.tech/documentation/payload/): A payload is a JSON object with additional data you can add to a vector.
- [Storage](https://qdrant.tech/documentation/storage/): Qdrant can use one of two options for storage, **In-memory** storage (Stores all vectors in RAM, has the highest speed since disk access is required only for persistence), or **Memmap** storage, (creates a virtual address space associated with the file on disk).
- Clients: the programming languages you can use to connect to Qdrant.

### 2.4 How do we get started?

The open source version of Qdrant is available as a docker image and it can be pulled and run from any machine with docker installed. If you don't have Docker installed in your PC you can follow the instructions in the official documentation [here](https://docs.docker.com/get-docker/). After that, open your terminal start by downloading the image with the following command.

```sh
docker pull qdrant/qdrant
```

Next, initialize Qdrant with the following command, and you should be good to go.

```sh
docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    qdrant/qdrant
```

You should see something similar to the following image.

![dockerqdrant](../images/docker_qdrant.png)

If you experience any issues during the start process, please let us know in our [discord channel here](https://qdrant.to/discord). We are always available and happy to help.

Now that you have Qdrant up and running, your next step is to pick a client to connect to it. We'll be using Python as it has the most mature data tools' ecosystem out there. So, let's start setting up our dev environment and getting the libraries we'll be using today.

```sh
# with mamba or conda
mamba env create -n my_env python=3.10
mamba activate my_env

# or with virtualenv
python -m venv venv
source venv/bin/activate

# install packages
pip install qdrant-client pandas numpy faker
```

After your have your environment ready, let's get started using Qdrant.

**Note:** At the time of writing, Qdrant supports Rust, GO, Python and TypeScript. We expect other programming languages to be added in the future.

## 3. Getting Started

The two modules we'll use the most are the `QdrantClient` and the `models` one. The former allows us to connect to Qdrant or it allows us to run an in-memory database by switching the parameter `location=` to `":memory:"` (this is a great feature for testing in a CI/CD pipeline). We'll start by instantiating our client using `host="localhost"` and `port=6333` (as it is the default port we used earlier with docker). You can also follow along with the `location=":memory:"` option commented out below.

In [1]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus

In [2]:
client = QdrantClient(host="localhost", port=6333)
client

In [3]:
# client = QdrantClient(location=":memory:")
# client

In OLTP and OLAP databases we call specific bundles of rows and columns **Tables**, but in vector databases, the rows are known as vectors, the columns are known as dimensions, and the combination of the two (plus some metadata) as **collections**.

In the same way in which we can create many tables in an OLTP or an OLAP database, we can create many collections in a vector database like Qdrant using one of its clients. The key difference to note is that when we create a collection in Qdrant, we need to specify the width of the collection (i.e. the length of the vector or amount of dimensions) beforehand with the parameter `size=...`, as well as the distance metric with the parameter `distance=...` (which can be changed later on).

The distances currently supported by Qdrant are:
- [**Cosine Similarity**](https://en.wikipedia.org/wiki/Cosine_similarity) - Cosine similarity is a way to measure how similar two things are. Think of it like a ruler that tells you how far apart two points are, but instead of measuring distance, it measures how similar two things are. It's often used with text to compare how similar two documents or sentences are to each other. The output of the cosine similarity ranges from 0 to 1, where 0 means the two things are completely dissimilar, and 1 means the two things are exactly the same. It's a straightforward and effective way to compare two things!
- [**Dot Product**](https://en.wikipedia.org/wiki/Dot_product) - The dot product similarity metric is another way of measuring how similar two things are, like cosine similarity. It's often used in machine learning and data science when working with numbers. The dot product similarity is calculated by multiplying the values in two sets of numbers, and then adding up those products. The higher the sum, the more similar the two sets of numbers are. So, it's like a scale that tells you how closely two sets of numbers match each other.
- [**Euclidean Distance**](https://en.wikipedia.org/wiki/Euclidean_distance) - Euclidean distance is a way to measure the distance between two points in space, similar to how we measure the distance between two places on a map. It's calculated by finding the square root of the sum of the squared differences between the two points' coordinates. This distance metric is commonly used in machine learning to measure how similar or dissimilar two data points are or, in other words, to understand how far apart they are.

Let's create our first collection and have the vectors be of size 100 with a distance set to **Cosine Similarity**. Please note that, at the time of writing, Qdrant only supports cosine similarity, dot product and euclidean distance for its distance metrics.

In [4]:
my_collection = "first_collection"

first_collection = client.recreate_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)
print(first_collection)

True


We can extract information related to the health of our collection by getting the collection. In addition, we can use this information for testing purposes, which can be very beneficial while in development mode.

In [5]:
collection_info = client.get_collection(collection_name=my_collection)
list(collection_info)

[('status', <CollectionStatus.GREEN: 'green'>),
 ('optimizer_status', <OptimizersStatusOneOf.OK: 'ok'>),
 ('vectors_count', 0),
 ('indexed_vectors_count', 0),
 ('points_count', 0),
 ('segments_count', 8),
 ('config',
  CollectionConfig(params=CollectionParams(vectors=VectorParams(size=100, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None)),
 ('payload_schema', {})]

In [6]:
assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == 0

There's a couple of things to notice from what we have done so far.
- The first is that when we initiated our docker image, we created a local directory called, `qdrant_storage`, and this is where all of our collections, plus their metadata, will be saved at. You can have a look at that directory in a *nix system with `tree qdrant_storage -L 2`, and something similar to the following output should come up for you.
    ```bash
    qdrant_storage
    ├── aliases
    │   └── data.json
    ├── collections
    │   └── my_first_collection
    └── raft_state
    ```
- The second is that we used `client.recreate_collection` and this command, as the name implies, can be used more than once to create new collections with or without the same name, so be careful no to recreate a collection that you did not intend to recreate. To create a brand new collection that cannot be recreated again, we would use `client.create_collection` instead.
- Our collection will hold vectors of 100 dimensions and the distance metric has been set to Cosine Similarity.

Now that we know how to create collections, let's create a bit of fake data and add some vectors to it.

### 3.1 Adding Points

The points are the central entity Qdrant operates with, and these contain records consisting of a vector, an optional `id` and an optional `payload` (which we'll talk more about in the next section).

The optional id can be represented by [unsigned integers](https://en.wikipedia.org/wiki/Integer_(computer_science)) or [UUID(https://en.wikipedia.org/wiki/Universally_unique_identifier)]s but, for our use case, we will use a straightforward range of numbers.

Let's us [NumPy](https://numpy.org/) to create a matrix of fake data containing 1,000 vectors and 100 dimensions and represent the values as `float64` numbers between -1 and 1. For simplicity, let's imagine that each of these vectors represents one of our favorite songs, and that each columns represents a unique characteristic of the artists/bands we love, for example, the tempo, the beats, the pitch of the voice of the singer(s), etc.

In [5]:
import numpy as np

In [6]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
type(data[0, 0]), data[:2, :20]

(numpy.float64,
 array([[-0.29745795, -0.78413275, -0.78550932, -0.75473754, -0.81713346,
          0.84760331,  0.99860196, -0.59123061,  0.14122297, -0.54089743,
         -0.25186033, -0.42010619, -0.5540755 ,  0.40097666,  0.24919242,
          0.65130027,  0.67931606,  0.698367  ,  0.12575482,  0.18913992],
        [-0.89031504,  0.53508302,  0.76607017, -0.69049807,  0.73844361,
          0.65496231, -0.34789025,  0.08822466,  0.34232025,  0.00671597,
         -0.97492937, -0.67219979,  0.62590413,  0.20612731, -0.69310484,
         -0.40526197, -0.41696134, -0.55274537, -0.99602328,  0.62102736]]))

Let's now create an index for our vectors.

In [7]:
index = list(range(len(data)))
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

Once the collection has been created, we can fill it in with the command `client.upsert()`. We'll need the collection's name and the appropriate uploading process from our `models` module, in this case, [`Batch`](https://qdrant.tech/documentation/points/#upload-points).

One thing to note is that Qdrant can only take in native Python iterables like lists and tuples. This is why you'll notice the `.tolist()` method attached to our numpy matrix,`data`, below.

In [8]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

We can retrieve specific points based on their ID (for example, artist X with ID 1000) and get some additional information from that result.

In [15]:
client.retrieve(
    collection_name=my_collection,
    ids=[100],
    with_vectors=True # the default is False
)

[Record(id=100, payload={}, vector=[0.16246888, -0.10363036, 0.14825524, 0.12942095, -0.16283034, 0.04708378, 0.089588456, 0.14510843, -0.07722532, 0.006147402, 0.15467958, -0.16686848, -0.0074468153, 0.13723059, 0.14898604, -0.020971628, 0.034934722, 0.055543285, 0.12387257, -0.08439057, 0.06172984, 0.16269302, -0.061226062, -0.15102349, 0.0014867382, -0.10153163, -0.16996586, -0.16822962, -0.15130003, 0.057790782, 0.089177035, 0.015628908, -0.029781206, 0.040165763, 0.09399984, -0.13046551, -0.053516887, -0.0540471, 0.10110339, -0.1542861, 0.057156898, 0.09246645, 0.026645578, -0.026840875, 0.079332285, -0.07057492, 0.15971132, -0.07860111, 0.054932095, -0.0070984163, -0.119582236, -0.030046042, -0.15760826, 0.017766219, -0.1444743, -0.14934336, 0.15682611, 0.0747199, 0.04417178, 0.12601678, 0.09686005, -0.032867387, -0.071897484, -0.14077185, -0.06922254, -0.15063968, 0.11458665, -0.08072215, -0.032622743, -0.06895638, 0.053184945, 0.101271, 0.08155548, 0.059098534, -0.0020898064, 0

We can also update our collection one point at a time, for example, as new data comes in.

In [16]:
def create_song():
    return np.random.uniform(low=-1.0, high=1.0, size=100).tolist()

In [17]:
client.upsert(
    collection_name=my_collection,
    points=[
        models.PointStruct(
            id=1000,
            vector=create_song(),
        )
    ]
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

We can also delete it in a straightforward fashion.

In [18]:
# this will show the amount of vectors BEFORE deleting the one we just created
client.count(
    collection_name=my_collection, 
    exact=True,
) 

CountResult(count=1001)

In [19]:
client.delete(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=[1000],
    ),
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
# this will show the amount of vectors AFTER deleting them
client.count(
    collection_name=my_collection, 
    exact=True,
)

CountResult(count=1000)

### 3.2 Payloads

Qdrant has incredible features on top of speed and reliability, and one of its most useful ones is without a doubt the ability to store additional information alongside the vectors. In Qdrant's terminology, this information is considered a payload and it is represented as JSON objects. With these payloads, not only can you get information back when you search in the database, but you can also filter your search by the parameters in the payload, and we'll see how in a second.

Imagine the fake vectors we created actually represented a song. If we were building a semantic search system for songs then, naturally, the things we would want to get back would be the song itself (or an URL to it), the artist, maybe the genre, and so on.

What we'll do here is to take advantage of a Python package call `faker` and create a bit of information to add to our payload and see how this functionality works.

In [9]:
from faker import Faker

In [10]:
fake_something = Faker()
fake_something.name()

'Richard Valencia'

For each vector, we'll create list of dictionaries containing the artist, the song, a url to the song, the year in which it was released, and the country where it originated from.

In [12]:
payload = []

for i in range(len(data)):
    payload.append(
        {
            "artist":   fake_something.name(),
            "song":     " ".join(fake_something.words()),
            "url_song": fake_something.url(),
            "year":     fake_something.year(),
            "country":  fake_something.country()
        }
    )

payload[:3]

[{'artist': 'John Love',
  'song': 'report hold election',
  'url_song': 'https://www.compton-watson.org/',
  'year': '1998',
  'country': 'Angola'},
 {'artist': 'Sara Salinas',
  'song': 'amount foot box',
  'url_song': 'https://wright.com/',
  'year': '2018',
  'country': 'Togo'},
 {'artist': 'Jenna Garrett',
  'song': 'world price office',
  'url_song': 'http://www.davis-rasmussen.com/',
  'year': '1990',
  'country': 'Kenya'}]

We can upsert our Points (ids, data, and payload), with the same `client.upsert()` method we used earlier, and we can retrieve any one song with the `client.retrieve()` method.

In [13]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
resutls = client.retrieve(
    collection_name=my_collection,
    ids=[10, 50, 100, 500],
    with_vectors=False
)

type(resutls), resutls

(list,
 [Record(id=10, payload={'artist': 'Joel Smith', 'country': 'Mozambique', 'song': 'more race of', 'url_song': 'https://crosby.org/', 'year': '1974'}, vector=None),
  Record(id=100, payload={'artist': 'Howard Rogers', 'country': 'Korea', 'song': 'number call put', 'url_song': 'http://deleon-baxter.net/', 'year': '2007'}, vector=None),
  Record(id=50, payload={'artist': 'Erika Jackson', 'country': 'Mozambique', 'song': 'over actually possible', 'url_song': 'http://www.fowler.net/', 'year': '2010'}, vector=None),
  Record(id=500, payload={'artist': 'Morgan Dennis', 'country': 'Greenland', 'song': 'leg full she', 'url_song': 'http://grant.com/', 'year': '1974'}, vector=None)])

We go back a list with of records where each elemen

In [29]:
resutls[0].payload

{'artist': 'Michael Aguilar',
 'country': 'Christmas Island',
 'song': 'me information range',
 'url_song': 'http://www.long-stevenson.com/',
 'year': '1994'}

Now that you know a little bit about the payload functionality of Qdrant, let's use it to search.

### 3.3 Search

Now that we have our vectors with an ID and a payload, we can explore a few of ways in which we can search for content when, in our use case, new music gets selected. Let's check it out.

Say, for example, that a new song comes in and our model immediately transforms it into a vector. Since we don't want a ridiculous amount of values back, let's limit the search to 10 points.

In [30]:
living_la_vida_loca = create_song()

In [32]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=3
)

[ScoredPoint(id=965, version=3, score=0.33937782, payload={'artist': 'Christopher Frye', 'country': 'Saint Helena', 'song': 'professional sea speak', 'url_song': 'https://www.lawrence.com/', 'year': '1978'}, vector=None),
 ScoredPoint(id=32, version=3, score=0.31084833, payload={'artist': 'Bryan Fields', 'country': 'Mongolia', 'song': 'policy such market', 'url_song': 'https://patrick.info/', 'year': '1982'}, vector=None),
 ScoredPoint(id=305, version=3, score=0.28512967, payload={'artist': 'Justin Carey', 'country': 'Mayotte', 'song': 'bad kid article', 'url_song': 'https://www.warren.org/', 'year': '2010'}, vector=None)]

Now imagine that we only want Australian songs recommended to us. For this, we can filter the query with a payload.

In [33]:
aussie_songs = models.Filter(
    must=[models.FieldCondition(key="country", match=models.MatchValue(value="Australia"))]
)
type(aussie_songs)

qdrant_client.http.models.models.Filter

In [34]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    limit=2
)

[ScoredPoint(id=202, version=3, score=0.13552207, payload={'artist': 'Stacy Mathis', 'country': 'Australia', 'song': 'none building mention', 'url_song': 'https://www.rodriguez-smith.com/', 'year': '2018'}, vector=None),
 ScoredPoint(id=618, version=3, score=0.049343247, payload={'artist': 'Susan Kerr', 'country': 'Australia', 'song': 'court necessary never', 'url_song': 'https://www.gray.biz/', 'year': '2006'}, vector=None)]

Lastly, say we want aussie songs but we don't care how new or old these songs are. Let's exclude points based on the year contained in the payload.

In [35]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    with_payload=models.PayloadSelectorExclude(exclude=["year"]),
    limit=5
)

[ScoredPoint(id=202, version=3, score=0.13552207, payload={'artist': 'Stacy Mathis', 'country': 'Australia', 'song': 'none building mention', 'url_song': 'https://www.rodriguez-smith.com/'}, vector=None),
 ScoredPoint(id=618, version=3, score=0.049343247, payload={'artist': 'Susan Kerr', 'country': 'Australia', 'song': 'court necessary never', 'url_song': 'https://www.gray.biz/'}, vector=None),
 ScoredPoint(id=115, version=3, score=0.03261761, payload={'artist': 'Scott Roberts', 'country': 'Australia', 'song': 'window resource newspaper', 'url_song': 'https://rice.info/'}, vector=None),
 ScoredPoint(id=314, version=3, score=0.012027343, payload={'artist': 'Mitchell Weaver', 'country': 'Australia', 'song': 'page feel music', 'url_song': 'http://obrien.com/'}, vector=None),
 ScoredPoint(id=816, version=3, score=-0.11323804, payload={'artist': 'Brandy Gonzalez', 'country': 'Australia', 'song': 'statement ball machine', 'url_song': 'http://brown-owens.com/'}, vector=None)]

As you can see, you can apply a wide-range of filtering methods to allows your users to take more control of the recommendations they are being served.

If you wanted to clear out the payload and upload a new for the same vectors, you can use `client.clear_payload()` as in the cell below.

In [36]:
client.clear_payload(
    collection_name=my_collection,
    points_selector=models.PointIdsList(
        points=index,
    )
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

## 5. Recommendations

That's it! You have now gone over a whirlwind tour of vector databases and are ready to tackle new challenges. 😎

## 5. Conclusion

In conclusion, we have explored a bit of the fascinating world of vector databases, natural language processing, transformers, and embeddings. In this tutorial we learned that (1) vector databases provide efficient storage and retrieval of high-dimensional vectors, making them ideal for similarity-based search tasks. (2) Natural language processing enables us to understand and process human language, opening up possibilities for different kinds of useful applications for digital technologies. (3) Transformers, with their attention mechanism, capture long-range dependencies in language and achieve incredible results in different tasks. Finally, embeddings encode words or sentences into dense vectors, capturing semantic relationships and enabling powerful language understanding.

By combining these technologies, we can unlock new levels of language understanding, information retrieval, and intelligent systems that continue to push the boundaries of what's possible in the realm of AI.

## 6. Resources

Here is a list with some resources that we found useful, and that helped with the development of this tutorial.

1. Books
    - [Natural Language Processing with Transformers](https://transformersbook.com/) by Lewis Tunstall, Leandro von Werra, and Thomas Wolf
    - [Natural Language Processing in Action, Second Edition](https://www.manning.com/books/natural-language-processing-in-action-second-edition) by Hobson Lane and Maria Dyshel
2. Articles
    - [Fine Tuning Similar Cars Search](https://qdrant.tech/articles/cars-recognition/)
    - [Q&A with Similarity Learning](https://qdrant.tech/articles/faq-question-answering/)
    - [Question Answering with LangChain and Qdrant without boilerplate](https://qdrant.tech/articles/langchain-integration/)
    - [Extending ChatGPT with a Qdrant-based knowledge base](https://qdrant.tech/articles/chatgpt-plugin/)
3. Videos
    - [Word Embedding and Word2Vec, Clearly Explained!!!](https://www.youtube.com/watch?v=viZrOnJclY0&ab_channel=StatQuestwithJoshStarmer) by StatQuest with Josh Starmer
    - [Word Embeddings, Bias in ML, Why You Don't Like Math, & Why AI Needs You](https://www.youtube.com/watch?v=25nC0n9ERq4&ab_channel=RachelThomas) by Rachel Thomas
4. Courses
    - [fast.ai Code-First Intro to Natural Language Processing](https://www.youtube.com/playlist?list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9)
    - [NLP Course by Hugging Face](https://huggingface.co/learn/nlp-course/chapter1/1)